Tweakreg sorts the imgaes into blocks and aligns them to a given catalogue, requirement ssbdev, DONT USE SSBREL
Remember to run sort_images_by_filter.py first to have the keyword FILTER added, as it might not exist in every HST fits file
See Documentation.docx for explaination of the parameters

In [ ]:
print '>>modules'
import argparse
import sys, os, re, time
from modules import move_im 
start=time.time()

#mode Jakob needs ssbdev
"""
tweakreg pictures, allign them to catalogue
make sure they have header keyword 'FILTER' (eg run sort_images_by_filter.py in jpy)
"""
def get_opt():
    parser=argparse.ArgumentParser()
    parser.add_argument('-trinput', type=str, default='flc.fits', help='uses all images which end with') 
    parser.add_argument('-VIS', type=str, default=[], nargs='*', help='give all visits, you want to run, standard is 01-20, example -VIS 01 04 10 15') #02, 14
    parser.add_argument('-FILTER', type=str, default=[], nargs='*', help='give all Filters (keyword filter) you want to run, otherwise all available Filters are taken, same syntax as -VIS') #F775W
    parser.add_argument('-mode', type=str, default='Jakob', help='only Jakob exists so far')
    parser.add_argument('-tr_refcat', type=str, default='30dor.0x900b.cat', help='give the catalogue (given in -dir(DONT USE IT DUE TO PROBLEMS EITH "." IN DIRECTORY NAME) or ./), MAKE SURE YOU ARE USING SSBDEV')
    parser.add_argument('-dir', type=str, default='./')
    
    args=parser.parse_args()
    return(args)

Modules are imported, Initialisation of argparse

In [ ]:
def tr_init(args, imdir):
    import drizzlepac
    print 'drizzlepac Version -> ', drizzlepac.__version__, '\n'
    from astropy.io import fits

    VIS=args.VIS
    print '\ninput: \nVIS', VIS
    FILTER=args.FILTER
    print 'FILTER', FILTER, '\n'
    mode=args.mode
    tr_refcat=args.tr_refcat
    cats=[]
    trinput=args.trinput
    if tr_refcat in os.listdir(imdir):
        print 'Using Cataloge %s'%(tr_refcat)
    else:
        for el in os.listdir(imdir):
            if re.search(tr_refcat, imdir):
                cats.append(el)
        if len(cats)!=1:
            print 'please specify catalogue'
            sys.exit(1)        

    start_filter=time.time()
    print '%.2f seconds since start, used for initialization'%(start_filter-start)
    if FILTER==[]:
        print('looking for Filter in all files, this might take some time')
        for el in os.listdir(imdir):
            if el.endswith(trinput):
                try:
                    hdulist=fits.open(imdir+el) 
                    new_filter=hdulist[0].header['FILTER']
                    if new_filter not in FILTER:
                        FILTER.append(new_filter)
                except:
                    print('could not open file %s'%(imdir+el))
            else:
                print('omitting %s')%(imdir+el)
    print 'It took %.2f seconds to find all filters'%(time.time()-start_filter)
    if VIS==[]:
        for i in xrange(1,21):
            if i<10: VIS.append('0'+str(i))
            else: VIS.append(str(i))
    return(VIS, FILTER, tr_refcat, trinput, mode)

Further Initialisation, checks if catalogue is found, otherwise Tweakreg would stilll work, but align the images to a self created calatogue based on the first one.

If no Filters are given as an input the program assumes all possibilities are to be considered, it goes through all images and tries to get the key word FILTER and creates its own list.

If no Visits are given the program creates its own list from 1 to 20:
[01, 02, 03, 04, 05, ..., 10, 11, 12,... 20]

Mode = Jakob
Go through the directory and find all files which match the visit number or any other string in the VIS list
If it is * all images are used
The list of names to be used is called items

Go through the items list, check if the filter of the file fits the filter being run
the list of all items to be used now is called final_items

Move all the final_items away in their directory imdir/FILTER_VISIT/

Runs Tweakreg on the outdir! If something goes wrong and you run the program again it might not find files in ypur directory, will not move them in the new directory and skip this VISIT and FILTER. It is never a mistake to make a backup copy!
Depending on how large the cataloue and the images this can take a while

All .png and .coo files are moved. They unfortunately belong to the output of tweakreg, eventhough the parameter clean is set to false

In [ ]:
def tr_rout(imdir, VIS, FILTER, tr_refcat, trinput, mode):
    print 'FILTER: ', FILTER
    print '>>running mode ', mode

    if mode=='Jakob':
        for VISno in VIS:
            time_visno=time.time()
            items=[]
            print '\n>>>Running VISIT ', VISno
            if VISno=='*':
                items=os.listdir(imdir)
            else:
                for el in os.listdir(imdir):
                    if re.search(str(VISno), el):
                        items.append(el)
            for FILTERno in FILTER:
                time_filter=time.time()
                final_items=[]
                print '\nRunning FILTER ', FILTERno
                for el in items:
                    try:
                        hdulist=fits.open(imdir+el)
                    except:
                        print '\tomitting %s'%(el)
                        continue
                    if hdulist[0].header['FILTER']==FILTERno:
                        print '\t',el
                        final_items.append(el)
                outdir=imdir+str(FILTERno)+'_'+str(VISno)+'/'
                for image in final_items:
                    try:
                        os.rename(imdir+image, outdir+image)
                    except OSError:
                        print '>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>\ncreating directory /%s/\n>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>'%(str(FILTERno)+'_'+str(VISno))
                        os.mkdir(outdir)
                        os.rename(imdir+image, outdir+image)
                print tr_refcat
                if not final_items==[]: drizzlepac.tweakreg.TweakReg(files=outdir+'*'+trinput, refimage=None, interactive=False, clean=True, updatewcs=True, updatehdr=True, wcsname='UPDATEWCS', refcat=imdir+tr_refcat)
                else: print 'No images for Visit {} and Filter {}'.format(VISno, FILTERno)
                print 'It took %.2f seconds to run tweakreg for this filter'%(time.time()-time_filter)
                move_im('png', imdir)
                move_im('coo', imdir)
            print 'It took %.2f seconds to run tweakreg for this visit'%(time.time()-time_visno)
    #write your own code here:
    #elif mode== "MyMode":
    #    Do stuff
    else:
        print 'please give an existing mode'

    move_im('log', imdir)
    return(True)

If the mode being used is not Jakob alternatives are looked at here, default can be set in the argparser
Moves tweakreg.log too, to clean up the directory

In [ ]:
if __name__=='__main__':
    args=get_opt()
    imdir=args.dir
    VIS, FILTER, tr_refcat, trinput, mode=tr_init(args, imdir)
    flag=tr_rout(imdir, VIS, FILTER, tr_refcat, trinput, mode)

Runs Script, first the init part, then the routine